# IMPORTING LIBRARYS

In [2]:
import pandas as pd
import numpy as np

import nltk
import string
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import warnings
warnings.filterwarnings("ignore")

# LOADING DATA 

In [3]:
movies_data=pd.read_csv(r"D:\minor_2\ml-latest\ml-latest\movies.csv")
ratings_data=pd.read_csv(r"D:\minor_2\ml-latest\ml-latest\ratings.csv")
tags_data=pd.read_csv(r"D:\minor_2\ml-latest\ml-latest\tags.csv")

# PRINTING DATASET

In [4]:
movies_data.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  58098 non-null  int64 
 1   title    58098 non-null  object
 2   genres   58098 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


In [6]:
ratings_data.head(5)

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [7]:
ratings_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 847.0 MB


In [8]:
tags_data.head(5)

,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195


In [9]:
tags_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1108997 entries, 0 to 1108996
Data columns (total 4 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   userId     1108997 non-null  int64 
 1   movieId    1108997 non-null  int64 
 2   tag        1108981 non-null  object
 3   timestamp  1108997 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 33.8+ MB


# REMOVING UNREQUIRED DATA

In [10]:
ratings_data.pop("timestamp")
tags_data.pop("timestamp")

0          1443148538
1          1443148532
2          1442169410
3          1442169421
4          1442615195
              ...    
1108992    1264379059
1108993    1264379058
1108994    1264379058
1108995    1264379058
1108996    1168346830
Name: timestamp, Length: 1108997, dtype: int64

# NORMALIZING DATA

In [11]:
movies_data["genres"]=movies_data["genres"].str.replace('|'," ")
movies_data.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
content_data=pd.merge(movies_data,tags_data,on="movieId",how="left")
content_data.head()

,movieId,title,genres,userId,tag
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1040.0,animated
1,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1040.0,buddy movie
2,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1040.0,Cartoon
3,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1040.0,cgi
4,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,1040.0,comedy


In [13]:
content_data.fillna("",inplace=True)
content_data=pd.DataFrame(content_data.groupby("movieId")["tag"].apply(lambda x: "%s" % " ".join(x)))
content_data.head()

,tag
movieId,
1,animated buddy movie Cartoon cgi comedy comput...
2,fantasy adapted from:book animals bad cgi base...
3,moldy old Ann Margaret Burgess Meredith Daryl ...
4,characters girl movie characters chick flick b...
5,steve martin steve martin pregnancy remake agi...


In [14]:
content_data=pd.merge(movies_data,content_data,on="movieId",how="left")
content_data.head()

,movieId,title,genres,tag
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,animated buddy movie Cartoon cgi comedy comput...
1,2,Jumanji (1995),Adventure Children Fantasy,fantasy adapted from:book animals bad cgi base...
2,3,Grumpier Old Men (1995),Comedy Romance,moldy old Ann Margaret Burgess Meredith Daryl ...
3,4,Waiting to Exhale (1995),Comedy Drama Romance,characters girl movie characters chick flick b...
4,5,Father of the Bride Part II (1995),Comedy,steve martin steve martin pregnancy remake agi...


In [15]:
content_data["metadata"]=content_data[["genres","tag"]].apply(lambda x: " ".join(x),axis=1)
content_data.head()

,movieId,title,genres,tag,metadata
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,animated buddy movie Cartoon cgi comedy comput...,Adventure Animation Children Comedy Fantasy an...
1,2,Jumanji (1995),Adventure Children Fantasy,fantasy adapted from:book animals bad cgi base...,Adventure Children Fantasy fantasy adapted fro...
2,3,Grumpier Old Men (1995),Comedy Romance,moldy old Ann Margaret Burgess Meredith Daryl ...,Comedy Romance moldy old Ann Margaret Burgess ...
3,4,Waiting to Exhale (1995),Comedy Drama Romance,characters girl movie characters chick flick b...,Comedy Drama Romance characters girl movie cha...
4,5,Father of the Bride Part II (1995),Comedy,steve martin steve martin pregnancy remake agi...,Comedy steve martin steve martin pregnancy rem...


# PREPROCESSING DATA

In [16]:
cleanup_re = re.compile('[^a-z]+')
def text_lower(sent):
    sent=str(sent)
    sent=sent.lower()
    sent=cleanup_re.sub(' ',sent).strip()
    return sent

In [17]:
lemmer = nltk.stem.WordNetLemmatizer()
def lematization(sent):
    sent=sent.split()
    temp=""
    for i in sent:
        flag=lemmer.lemmatize(i)
        temp=temp+" "+flag
    return temp

In [18]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def remove_stopwords(sent):
    word_tokens=sent.split()
    temp=""
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    for i in filtered_sentence:
        temp=temp+" "+i
    return temp

In [19]:
content_data["clean_data"]=content_data["metadata"].apply(text_lower)
content_data["clean_data"]=content_data["clean_data"].apply(lematization)
content_data["clean_data"]=content_data["clean_data"].apply(remove_stopwords)
content_data.head()

,movieId,title,genres,tag,metadata,clean_data
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,animated buddy movie Cartoon cgi comedy comput...,Adventure Animation Children Comedy Fantasy an...,adventure animation child comedy fantasy anim...
1,2,Jumanji (1995),Adventure Children Fantasy,fantasy adapted from:book animals bad cgi base...,Adventure Children Fantasy fantasy adapted fro...,adventure child fantasy fantasy adapted book ...
2,3,Grumpier Old Men (1995),Comedy Romance,moldy old Ann Margaret Burgess Meredith Daryl ...,Comedy Romance moldy old Ann Margaret Burgess ...,comedy romance moldy old ann margaret burgess...
3,4,Waiting to Exhale (1995),Comedy Drama Romance,characters girl movie characters chick flick b...,Comedy Drama Romance characters girl movie cha...,comedy drama romance character girl movie cha...
4,5,Father of the Bride Part II (1995),Comedy,steve martin steve martin pregnancy remake agi...,Comedy steve martin steve martin pregnancy rem...,comedy steve martin steve martin pregnancy re...


# MODEL BUILDING

In [1]:
from fuzzywuzzy import process

C:\Users\DELL\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [53]:

idx=process.extractOne("toy story",content_data["title"])[2]
print(idx)
print("movie selected : ",content_data["title"][idx])
print("searching for recommendation....")
para=content_data["clean_data"][idx]

0
movie selected :  Toy Story (1995)
searching for recommendation....


In [54]:
watched_movie=para
watched_movie=text_lower(watched_movie)
watched_movie=lematization(watched_movie)
watched_movie=remove_stopwords(watched_movie)

In [55]:
content_data.loc[len(content_data.index)] = [0,"A","a","A","a",watched_movie] 

In [56]:
tfid=TfidfVectorizer(stop_words='english')
tfidf= tfid.fit_transform(content_data["clean_data"])

In [57]:
vals = cosine_similarity(tfidf[-1],tfidf)
idx = vals.argsort()[0][-3]
flat=vals.flatten()
flat.sort()
req_tfidf= flat[-2]

# PREDICTION

In [58]:
for i in range(-2,-13,-1):
    idx = vals.argsort()[0][i]
    print(content_data.iloc[idx]["title"])

Toy Story (1995)
Toy Story 2 (1999)
Bug's Life, A (1998)
Monsters, Inc. (2001)
Toy Story 3 (2010)
Lou (2017)
Finding Nemo (2003)
Finding Dory (2016)
Tin Toy (1988)
The Adventures of André and Wally B. (1984)
Incredibles, The (2004)


In [59]:
content_data.drop(content_data.tail(1).index,inplace=True)